# 📊 Análisis Exploratorio de Datos (EDA)
## FlightOnTime - Predicción de Retrasos de Vuelos

---

### Objetivos:
1. Cargar y explorar el dataset de vuelos
2. Crear la variable objetivo `is_delayed`
3. Analizar la distribución de retrasos
4. Identificar patrones por aerolínea, hora, día dela semana
5. Generar visualizaciones clave

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Configuración de pandas
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

print("✓ Librerías importadas correctamente")

## 1️⃣ Cargar el Dataset

In [ ]:
# Importar módulos del proyecto
import sys
sys.path.append('..')  # Para importar desde src/

from src.config import get_raw_data_path, DELAY_THRESHOLD, TARGET_COLUMN
from src.preprocessing import load_flight_data, normalize_column_names, create_target_variable

print("✓ Módulos del proyecto importados")

In [ ]:
# Cargar datos
raw_path = get_raw_data_path()
print(f"📂 Cargando datos desde: {raw_path}\n")

# Cargar con límite de filas para evitar problemas de memoria en Colab
df = load_flight_data(raw_path, sample_size=None)  # None = cargar todo

print(f"\n✓ Datos cargados: {df.shape[0]:,} registros, {df.shape[1]} columnas")

## 2️⃣ Exploración Inicial

In [ ]:
# Primeras filas
print("📋 Primeras 10 filas del dataset:\n")
df.head(10)

In [ ]:
# Información general
print("📊 Información del dataset:\n")
df.info()

In [ ]:
# Estadísticas descriptivas
print("📈 Estadísticas descriptivas:\n")
df.describe()

In [ ]:
# Valores nulos
print("🔍 Valores nulos por columna:\n")
null_counts = df.isnull().sum()
null_pct = (null_counts / len(df) * 100).round(2)

null_summary = pd.DataFrame({
    'Nulos': null_counts,
    'Porcentaje': null_pct
})

null_summary[null_summary['Nulos'] > 0].sort_values('Nulos', ascending=False)

## 3️⃣ Crear Variable Objetivo

In [ ]:
# Normalizar nombres de columnas
df = normalize_column_names(df)

# Crear variable objetivo: is_delayed = 1 si dep_delay > 15 minutos
df = create_target_variable(df)

print(f"\n✓ Variable objetivo '{TARGET_COLUMN}' creada con éxito")

In [ ]:
# Distribución de la variable objetivo
print("📊 Distribución del target:\n")
target_dist = df[TARGET_COLUMN].value_counts().sort_index()
target_pct = df[TARGET_COLUMN].value_counts(normalize=True).sort_index() * 100

dist_summary = pd.DataFrame({
    'Cantidad': target_dist,
    'Porcentaje': target_pct.round(2)
})

dist_summary.index = ['Puntual (0)', 'Retrasado (1)']
print(dist_summary)

print(f"\n💡 Balance de clases: {target_pct.min():.1f}% / {target_pct.max():.1f}%")

In [ ]:
# Visualización de la distribución del target
fig, ax = plt.subplots(figsize=(10, 6))

colors = ['#2ecc71', '#e74c3c']
target_dist.plot(kind='bar', ax=ax, color=colors, edgecolor='black', linewidth=1.5)

ax.set_xlabel('Clase', fontsize=12, fontweight='bold')
ax.set_ylabel('Cantidad de Vuelos', fontsize=12, fontweight='bold')
ax.set_title(f'Distribución de la Variable Objetivo\n(Umbral: {DELAY_THRESHOLD} minutos)', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xticklabels(['Puntual (0)', 'Retrasado (1)'], rotation=0)

# Añadir etiquetas de valores
for i, v in enumerate(target_dist):
    ax.text(i, v + len(df)*0.01, f'{v:,}\n({target_pct.iloc[i]:.1f}%)', 
            ha='center', va='bottom', fontweight='bold')

ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('../outputs/figures/target_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Gráfica guardada: outputs/figures/target_distribution.png")

## 4️⃣ Análisis de Retrasos

In [ ]:
# Distribución de retrasos (solo vuelos con retraso > 0)
if 'dep_delay' in df.columns:
    delayed_flights = df[df['dep_delay'] > 0]['dep_delay']
    
    print(f"📊 Estadísticas de retrasos (solo vuelos retrasados):\n")
    print(f"  Media: {delayed_flights.mean():.2f} minutos")
    print(f"  Mediana: {delayed_flights.median():.2f} minutos")
    print(f"  Desviación estándar: {delayed_flights.std():.2f} minutos")
    print(f"  Mínimo: {delayed_flights.min():.2f} minutos")
    print(f"  Máximo: {delayed_flights.max():.2f} minutos")
    print(f"  Percentil 75: {delayed_flights.quantile(0.75):.2f} minutos")
    print(f"  Percentil 90: {delayed_flights.quantile(0.90):.2f} minutos")
    print(f"  Percentil 95: {delayed_flights.quantile(0.95):.2f} minutos")

In [ ]:
# Histograma de retrasos
if 'dep_delay' in df.columns:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Limitar a retrasos razonables para mejor visualización
    delays_to_plot = df[df['dep_delay'].between(0, 180)]['dep_delay']
    
    ax.hist(delays_to_plot, bins=60, color='#3498db', edgecolor='black', alpha=0.7)
    ax.axvline(x=DELAY_THRESHOLD, color='red', linestyle='--', linewidth=2, 
               label=f'Umbral de retraso ({DELAY_THRESHOLD} min)')
    
    ax.set_xlabel('Retraso (minutos)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
    ax.set_title('Distribución de Retrasos en Salidas\n(0-180 minutos)', 
                 fontsize=14, fontweight='bold', pad=20)
    ax.legend(fontsize=11)
    ax.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../outputs/figures/delay_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("💾 Gráfica guardada: outputs/figures/delay_distribution.png")

## 5️⃣ Análisis por Aerolínea

In [ ]:
# Analizar si existe columna de aerolínea
airline_col = None
for col in ['airline', 'carrier', 'op_carrier']:
    if col in df.columns:
        airline_col = col
        break

if airline_col:
    # Tasa de retraso por aerolínea
    airline_delay = df.groupby(airline_col)[TARGET_COLUMN].agg(['mean', 'count']).reset_index()
    airline_delay.columns = ['Aerolínea', 'Tasa_Retraso', 'Total_Vuelos']
    airline_delay['Tasa_Retraso'] = (airline_delay['Tasa_Retraso'] * 100).round(2)
    airline_delay = airline_delay[airline_delay['Total_Vuelos'] >= 100]  # Filtrar aerolíneas con pocos vuelos
    airline_delay = airline_delay.sort_values('Tasa_Retraso', ascending=False).head(15)
    
    print(f"🛫 Top 15 Aerolíneas con Mayor Tasa de Retraso (mín 100 vuelos):\n")
    print(airline_delay.to_string(index=False))
else:
    print("⚠️ No se encontró columna de aerolínea")

In [ ]:
# Visualización por aerolínea
if airline_col:
    fig, ax = plt.subplots(figsize=(14, 8))
    
    colors = plt.cm.RdYlGn_r(np.linspace(0.3, 0.9, len(airline_delay)))
    bars = ax.barh(airline_delay['Aerolínea'], airline_delay['Tasa_Retraso'], 
                   color=colors, edgecolor='black', linewidth=1.2)
    
    ax.set_xlabel('Tasa de Retraso (%)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Aerolínea', fontsize=12, fontweight='bold')
    ax.set_title('Tasa de Retraso por Aerolínea\n(Top 15 - Mínimo 100 vuelos)', 
                 fontsize=14, fontweight='bold', pad=20)
    
    # Añadir etiquetas de valores
    for i, (idx, row) in enumerate(airline_delay.iterrows()):
        ax.text(row['Tasa_Retraso'] + 0.5, i, 
                f"{row['Tasa_Retraso']:.1f}% ({row['Total_Vuelos']:,} vuelos)",
                va='center', fontsize=9)
    
    ax.grid(axis='x', alpha=0.3)
    ax.invert_yaxis()
    plt.tight_layout()
    plt.savefig('../outputs/figures/delay_by_airline.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("💾 Gráfica guardada: outputs/figures/delay_by_airline.png")

## 6️⃣ Análisis Temporal

In [ ]:
# Extraer features temporales
from src.features import extract_temporal_features, add_time_slots

# Buscar columna de fecha
date_col = None
for col in ['fl_date', 'date', 'flight_date']:
    if col in df.columns:
        date_col = col
        break

if date_col:
    df = extract_temporal_features(df, date_col)
    df = add_time_slots(df)
    print("\n✓ Features temporales extraídas")
else:
    print("⚠️ No se encontró columna de fecha")

In [ ]:
# Retrasos por hora del día
if 'hour' in df.columns:
    hour_delay = df.groupby('hour')[TARGET_COLUMN].agg(['mean', 'count']).reset_index()
    hour_delay.columns = ['Hora', 'Tasa_Retraso', 'Total_Vuelos']
    hour_delay['Tasa_Retraso'] = (hour_delay['Tasa_Retraso'] * 100).round(2)
    
    print("🕐 Tasa de Retraso por Hora del Día:\n")
    print(hour_delay.to_string(index=False))

In [ ]:
# Visualización por hora
if 'hour' in df.columns:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    ax.plot(hour_delay['Hora'], hour_delay['Tasa_Retraso'], 
            marker='o', linewidth=2.5, markersize=8, color='#e74c3c')
    
    ax.set_xlabel('Hora del Día', fontsize=12, fontweight='bold')
    ax.set_ylabel('Tasa de Retraso (%)', fontsize=12, fontweight='bold')
    ax.set_title('Tasa de Retraso por Hora del Día', fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(range(0, 24))
    ax.grid(alpha=0.3)
    
    # Sombrear franjas horarias
    ax.axvspan(0, 6, alpha=0.1, color='blue', label='Madrugada')
    ax.axvspan(6, 12, alpha=0.1, color='yellow', label='Mañana')
    ax.axvspan(12, 18, alpha=0.1, color='orange', label='Tarde')
    ax.axvspan(18, 24, alpha=0.1, color='purple', label='Noche')
    
    ax.legend(loc='upper left', fontsize=10)
    plt.tight_layout()
    plt.savefig('../outputs/figures/delay_by_hour.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("💾 Gráfica guardada: outputs/figures/delay_by_hour.png")

In [ ]:
# Retrasos por día de la semana
if 'day_of_week' in df.columns:
    dow_delay = df.groupby('day_of_week')[TARGET_COLUMN].agg(['mean', 'count']).reset_index()
    dow_delay.columns = ['Día', 'Tasa_Retraso', 'Total_Vuelos']
    dow_delay['Tasa_Retraso'] = (dow_delay['Tasa_Retraso'] * 100).round(2)
    dow_delay['Día_Nombre'] = dow_delay['Día'].map({
        0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 3: 'Jueves',
        4: 'Viernes', 5: 'Sábado', 6: 'Domingo'
    })
    
    print("📅 Tasa de Retraso por Día de la Semana:\n")
    print(dow_delay[['Día_Nombre', 'Tasa_Retraso', 'Total_Vuelos']].to_string(index=False))
    
    # Visualización
    fig, ax = plt.subplots(figsize=(12, 6))
    
    colors = ['#3498db'] * 5 + ['#e74c3c', '#e74c3c']  # Destacar fin de semana
    ax.bar(dow_delay['Día_Nombre'], dow_delay['Tasa_Retraso'], 
           color=colors, edgecolor='black', linewidth=1.5)
    
    ax.set_xlabel('Día de la Semana', fontsize=12, fontweight='bold')
    ax.set_ylabel('Tasa de Retraso (%)', fontsize=12, fontweight='bold')
    ax.set_title('Tasa de Retraso por Día de la Semana', fontsize=14, fontweight='bold', pad=20)
    ax.grid(axis='y', alpha=0.3)
    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('../outputs/figures/delay_by_day_of_week.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("💾 Gráfica guardada: outputs/figures/delay_by_day_of_week.png")

## 7️⃣ Conclusiones del EDA

### Hallazgos Principales:

1. **Balance de Clases**: El dataset presenta [completar según resultados] de vuelos retrasados vs puntuales

2. **Patrones Temporales**:
   - Las horas [completar] presentan mayor tasa de retrasos
   - Los días [completar] tienen más retrasos

3. **Aerolíneas**: Existe variabilidad significativa en las tasas de retraso entre aerolíneas

4. **Próximos Pasos**:
   - Ingeniería de características (ya iniciada)
   - Entrenamiento de modelos
   - Evaluación y selección del mejor modelo

---

**Notebook completado** ✅  
Continuar con: `01_train_model.ipynb`